In [2]:
# DATASET DE REGRESSÃO
from sklearn.datasets import fetch_california_housing, load_diabetes, make_regression, load_linnerud, fetch_openml
import pandas as pd
from IPython.display import clear_output
import numpy as np

# Função para carregar e transformar os datasets de regressão em problemas binários
def carregar_dataset_regressao_binario(nome_dataset):
    if nome_dataset == 'california_housing_binary':
        data = fetch_california_housing()
        X = pd.DataFrame(data.data, columns=data.feature_names)
        y = pd.qcut(data.target, q=2, labels=[0, 1]).astype(int)  # Dividindo pela mediana
        class_names = ['Low Price', 'High Price']

    elif nome_dataset == 'ames_housing_binary':
        data = fetch_openml(name='house_prices', as_frame=True)
        X = data.data.select_dtypes(include=[np.number])  # Seleciona apenas colunas numéricas
        y = pd.qcut(data.target, q=2, labels=[0, 1]).astype(int)  # Binarizando pela mediana
        class_names = ['Low Price', 'High Price']

    elif nome_dataset == 'diabetes_binary':
        data = load_diabetes()
        X = pd.DataFrame(data.data, columns=data.feature_names)
        y = pd.qcut(data.target, q=2, labels=[0, 1]).astype(int)  # Transformando pela mediana
        class_names = ['Low Progression', 'High Progression']

    elif nome_dataset == 'synthetic_regression_binary':
        X, y_cont = make_regression(n_samples=500, n_features=10, noise=0.1)
        y = pd.qcut(y_cont, q=2, labels=[0, 1]).astype(int)  # Dividindo os valores de y pela mediana
        X = pd.DataFrame(X, columns=[f"feature_{i}" for i in range(X.shape[1])])
        class_names = ['Low Output', 'High Output']

    elif nome_dataset == 'linnerud_binary':
        data = load_linnerud()
        X = pd.DataFrame(data.data, columns=data.feature_names)
        y = pd.qcut(data.target[:, 0], q=2, labels=[0, 1]).astype(int)  # Convertendo pela primeira coluna
        class_names = ['Low Fitness', 'High Fitness']

    elif nome_dataset == 'carbon_nanotubes_binary':
        # Gerando um dataset fixo para evitar problemas de aleatoriedade
        np.random.seed(42)
        X = pd.DataFrame(np.random.rand(500, 6), columns=[f"feature_{i}" for i in range(6)])
        y = np.where(X['feature_0'] > X['feature_0'].median(), 1, 0)  # Binarização com base em uma coluna
        class_names = ['Low Value', 'High Value']

    elif nome_dataset == 'random_dataset_1':
        X, y_cont = make_regression(n_samples=500, n_features=8, noise=0.5, random_state=1)
        y = pd.qcut(y_cont, q=2, labels=[0, 1]).astype(int)
        X = pd.DataFrame(X, columns=[f"feature_{i}" for i in range(X.shape[1])])
        class_names = ['Low Outcome', 'High Outcome']

    elif nome_dataset == 'random_dataset_2':
        X, y_cont = make_regression(n_samples=500, n_features=5, noise=1.0, random_state=2)
        y = pd.qcut(y_cont, q=2, labels=[0, 1]).astype(int)
        X = pd.DataFrame(X, columns=[f"feature_{i}" for i in range(X.shape[1])])
        class_names = ['Low Output', 'High Output']

    elif nome_dataset == 'random_dataset_3':
        X, y_cont = make_regression(n_samples=500, n_features=7, noise=0.3, random_state=3)
        y = pd.qcut(y_cont, q=2, labels=[0, 1]).astype(int)
        X = pd.DataFrame(X, columns=[f"feature_{i}" for i in range(X.shape[1])])
        class_names = ['Low Value', 'High Value']

    elif nome_dataset == 'random_dataset_4':
        X, y_cont = make_regression(n_samples=500, n_features=6, noise=0.7, random_state=4)
        y = pd.qcut(y_cont, q=2, labels=[0, 1]).astype(int)
        X = pd.DataFrame(X, columns=[f"feature_{i}" for i in range(X.shape[1])])
        class_names = ['Low Result', 'High Result']

    else:
        raise ValueError("Nome do dataset não reconhecido. Escolha um dataset válido.")
    
    return X, y, class_names

# Menu de seleção de datasets de regressão
menu = '''
|  ************************* MENU ***************************  |
|  0 - california_housing_binary   |  1 - ames_housing_binary    |
|  2 - diabetes_binary             |  3 - synthetic_regression_binary |
|  4 - linnerud_binary             |  5 - carbon_nanotubes_binary |
|  6 - random_dataset_1            |  7 - random_dataset_2        |
|  8 - random_dataset_3            |  9 - random_dataset_4        |
|  Q - SAIR                                                   |
|-------------------------------------------------------------|
'''

# Loop do menu com validação adequada
while True:
    print(menu)
    opcao = input("Digite o número do dataset ou 'Q' para sair: ").upper().strip()

    if opcao == 'Q':
        clear_output()
        print("Você escolheu sair.")
        break
    elif opcao.isdigit() and 0 <= int(opcao) <= 9:
        nomes_datasets = [
            'california_housing_binary', 'ames_housing_binary', 'diabetes_binary', 'synthetic_regression_binary',
            'linnerud_binary', 'carbon_nanotubes_binary', 'random_dataset_1', 'random_dataset_2',
            'random_dataset_3', 'random_dataset_4'
        ]
        nome_dataset = nomes_datasets[int(opcao)]
        clear_output()
        print(f"Dataset '{nome_dataset}' escolhido.")
        try:
            X, y, class_names = carregar_dataset_regressao_binario(nome_dataset)
            print(f"Dataset {nome_dataset} carregado com sucesso.")
            print("Classes:", class_names)
            print("Amostras:", X.shape[0], "| Atributos:", X.shape[1])
            break
        except Exception as e:
            print(f"Erro ao processar o dataset: {e}")
    else:
        clear_output()
        print("Opção inválida. Por favor, escolha um número do menu ou 'Q' para sair.")


Dataset 'california_housing_binary' escolhido.
Dataset california_housing_binary carregado com sucesso.
Classes: ['Low Price', 'High Price']
Amostras: 20640 | Atributos: 8


In [17]:
# DATASETS DE CLASSIFICAÇÃO
from sklearn.datasets import load_iris, load_wine, load_breast_cancer, load_digits
import pandas as pd
from IPython.display import clear_output

# Função para carregar os datasets
def carregar_dataset(nome_dataset):
    if nome_dataset == 'iris':
        data = load_iris()
        X, y = pd.DataFrame(data.data, columns=data.feature_names), data.target
        class_names = data.target_names
    
    elif nome_dataset == 'wine':
        data = load_wine()
        X, y = pd.DataFrame(data.data, columns=data.feature_names), data.target
        class_names = data.target_names
    
    elif nome_dataset == 'breast_cancer':
        data = load_breast_cancer()
        X, y = pd.DataFrame(data.data, columns=data.feature_names), data.target
        class_names = data.target_names
    
    elif nome_dataset == 'digits':
        data = load_digits()
        X, y = pd.DataFrame(data.data, columns=[f"pixel_{i}" for i in range(data.data.shape[1])]), data.target
        class_names = [str(i) for i in range(10)]
    
    elif nome_dataset == 'banknote_authentication':
        data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/00267/data_banknote_authentication.txt", header=None)
        X = data.iloc[:, :-1]
        y = data.iloc[:, -1]
        class_names = ['Legitimate', 'Forgery']
    
    elif nome_dataset == 'wine_quality':
        data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv", sep=';')
        X = data.drop(columns=['quality'])
        y = data['quality']
        class_names = sorted(y.unique().tolist())
    
    elif nome_dataset == 'heart_disease':
        data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data", header=None, na_values="?")
        data = data.dropna()  # Remove valores ausentes
        X = data.iloc[:, :-1]
        y = data.iloc[:, -1]
        class_names = sorted(y.unique().tolist())
    
    elif nome_dataset == 'parkinsons':
        data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/parkinsons/parkinsons.data")
        X = data.drop(columns=['status', 'name'])
        y = data['status']
        class_names = ['Healthy', 'Parkinsons']
    
    elif nome_dataset == 'car_evaluation':
        data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data", header=None)
        data.columns = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
        X = pd.get_dummies(data.drop(columns=['class']))
        y = data['class'].factorize()[0]
        class_names = data['class'].unique()
    
    elif nome_dataset == 'diabetes_binary':
        data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/diabetes/diabetes_data_upload.csv")
        X = pd.get_dummies(data.drop(columns=['class']))
        y = data['class'].apply(lambda x: 1 if x == 'Positive' else 0)
        class_names = ['Negative', 'Positive']
    
    else:
        raise ValueError("Nome do dataset não reconhecido. Escolha um dataset válido.")
    
    return X, y, class_names

# Menu de seleção de datasets
menu = '''
|  ************************* MENU ***************************  |
|  0 - iris                     |  1 - wine                     |
|  2 - breast_cancer            |  3 - digits                  |
|  4 - banknote_authentication  |  5 - wine_quality           |
|  6 - heart_disease            |  7 - parkinsons             |
|  8 - car_evaluation           |  9 - diabetes_binary        |
|  Q - SAIR                                                |
|-------------------------------------------------------------|
'''

# Exibe o menu e solicita uma escolha
print(menu)
opcao = input("Digite o número do dataset ou 'Q' para sair: ").upper().strip()

# Processa a opção selecionada
while True:
    if opcao == 'Q':
        clear_output()
        print("Você escolheu sair.")
        break
    elif opcao.isdigit() and 0 <= int(opcao) <= 9:
        nomes_datasets = [
            'iris', 'wine', 'breast_cancer', 'digits', 'banknote_authentication',
            'wine_quality', 'heart_disease', 'parkinsons', 'car_evaluation', 'diabetes_binary'
        ]
        nome_dataset = nomes_datasets[int(opcao)]
        clear_output()
        print(f"Dataset '{nome_dataset}' escolhido.")
        try:
            X, y, class_names = carregar_dataset(nome_dataset)
            print(f"Dataset {nome_dataset} carregado com sucesso.")
            print("Classes:", class_names)
            print("Amostras:", X.shape[0], "| Atributos:", X.shape[1])
            break
        except Exception as e:
            print(f"Erro ao processar o dataset: {e}")
    else:
        clear_output()
        print(menu)
        print("Opção inválida. Por favor, escolha um número do menu ou 'Q' para sair.")
        opcao = input("Digite o número do dataset ou 'Q' para sair: ").upper().strip()


Dataset 'iris' escolhido.
Dataset iris carregado com sucesso.
Classes: ['setosa' 'versicolor' 'virginica']
Amostras: 150 | Atributos: 4


In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import pandas as pd

# Função para transformar o dataset em um problema binário (classe 0 contra as outras)
def transformar_problema_binario(y, classe_0):
    return [1 if label == classe_0 else 0 for label in y]

def analisar_instancias(X, y, class_names, classe_0=0, instancia_para_analisar=None):
    global TUDO
    if not isinstance(X, pd.DataFrame):
        X = pd.DataFrame(X, columns=[f"feature_{i}" for i in range(X.shape[1])])

    # Transforma o problema em binário
    y_binario = transformar_problema_binario(y, classe_0)

    # Divide o dataset em treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y_binario, test_size=0.2, random_state=42)
    
    # Treina o modelo
    modelo = LogisticRegression(max_iter=200)
    modelo.fit(X_train, y_train)

    # Obtém os nomes das features
    feature_names = X.columns.tolist()  
    
    # Seleciona as instâncias para análise
    num_instancias = len(X_test)
    instancias_para_analisar = range(num_instancias) if instancia_para_analisar is None else [instancia_para_analisar]
    
    TUDO = []
    # Loop para analisar cada instância selecionada
    for idx in instancias_para_analisar:
        Vs = X_test.iloc[idx].to_dict()
        instancia_test = X_test.iloc[[idx]]

        # Calcula `gamma_A` usando `decision_function`
        gamma_A = modelo.decision_function(instancia_test)[0]
        
        # Cálculo do valor delta para cada feature
        delta = []
        w = modelo.coef_[0]
        for i, feature in enumerate(feature_names):
            if w[i] < 0:
                delta.append((Vs[feature] - X[feature].max()) * w[i])
            else:
                delta.append((Vs[feature] - X[feature].min()) * w[i])

        # Calcula R
        R = sum(delta) - gamma_A
        
        # Computa a PI-explicação para a instância atual usando nomes das features
        Xpl = one_explanation(Vs, delta, R, feature_names, modelo, instancia_test, X)
        # Imprime os resultados
        classe_verdadeira = y_test[idx]
        print(f"\nInstância {idx}:")
        print(f"Classe verdadeira (binária): {classe_verdadeira}")
        print(f"PI-Explicação: ")
        
        TUDO.append(Xpl)

        for item in Xpl:
            print(f"- {item}")
        
        
        if not Xpl:
            print('_No-PI-explanation_'*3)
        

# Função para calcular a PI-explicação e incluir os intervalos de valores mínimos e máximos que garantem a classe
def one_explanation(Vs, delta, R, feature_names, modelo, instancia_test, X):
    limiar_delta = np.percentile(np.abs(delta), 25)  # Pega o percentil 25 dos deltas
    Xpl = []
    delta_sorted = sorted(enumerate(delta), key=lambda x: abs(x[1]), reverse=True)
    R_atual = R
    Idx = 0
    
    while R_atual >= 0 and Idx < len(delta_sorted):
        sorted_idx, delta_value = delta_sorted[Idx]
        feature = feature_names[sorted_idx]
        feature_value = Vs[feature]

        if abs(delta_value) < limiar_delta:  # Descarta deltas muito pequenos
           break

        # Encontra os valores mínimo e máximo para manter a classe usando perturbação
        #min_val, max_val = encontrar_intervalo_perturbacao(modelo, instancia_test, feature, feature_value, classe_desejada=1, X=X)

        # Adiciona a feature com o valor da instância e o intervalo mínimo/máximo que mantém a classe
        Xpl.append(f"{feature} - {feature_value} ")
        R_atual -= delta_value
        Idx += 1
    
    return Xpl

# Função para encontrar o intervalo de perturbação para manter a classe, considerando limites de X
def encontrar_intervalo_perturbacao(modelo, instancia, feature, valor_original, classe_desejada, X, passo=0.1, max_iter=50):
    # Define os valores mínimo e máximo baseados nos dados de entrada
    min_val_data = X[feature].min()
    max_val_data = X[feature].max()
    
    # Inicializa os valores mínimo e máximo com o valor da instância
    min_val, max_val = valor_original, valor_original
    
    # Perturba negativamente
    for _ in range(max_iter):
        min_val -= passo
        if min_val < min_val_data:
            min_val = min_val_data
            break
        instancia_perturbada = instancia.copy()
        instancia_perturbada[feature] = min_val
        predicao = modelo.predict(instancia_perturbada)
        if predicao[0] != classe_desejada:
            min_val += passo
            break

    # Perturba positivamente
    for _ in range(max_iter):
        max_val += passo
        if max_val > max_val_data:
            max_val = max_val_data
            break
        instancia_perturbada = instancia.copy()
        instancia_perturbada[feature] = max_val
        predicao = modelo.predict(instancia_perturbada)
        if predicao[0] != classe_desejada:
            max_val -= passo
            break

    return min_val, max_val

# Exemplo de uso (substitua X e y pelos dados adequados)
analisar_instancias(X, y, class_names, classe_0=0)
# Crie um dicionário para armazenar a contagem de cada feature
contagem_features = {}

# Itere sobre cada item da lista TUDO
for item in TUDO:
    # Verifique se o item é uma lista
    if isinstance(item, list):
        # Itere sobre cada item da lista
        for feature in item:
            # Extraia o nome da feature
            nome_feature = feature.split(" - ")[0]

            # Verifique se a feature já está no dicionário
            if nome_feature in contagem_features:
                # Incremente a contagem
                contagem_features[nome_feature] += 1
            else:
                # Adicione a feature ao dicionário com contagem 1
                contagem_features[nome_feature] = 1

# Imprima quantidade em que as features aparecem na explicacão em ordem
for nome_feature, contagem in contagem_features.items():
    print(f"Feature: {nome_feature} Contagem: {contagem}")


Instância 0:
Classe verdadeira (binária): 0
PI-Explicação: 
- petal length (cm) - 4.7 
- petal width (cm) - 1.2 
- sepal length (cm) - 6.1 

Instância 1:
Classe verdadeira (binária): 1
PI-Explicação: 
- petal length (cm) - 1.7 
- petal width (cm) - 0.3 

Instância 2:
Classe verdadeira (binária): 0
PI-Explicação: 
- sepal width (cm) - 2.6 
- petal width (cm) - 2.3 
- sepal length (cm) - 7.7 

Instância 3:
Classe verdadeira (binária): 0
PI-Explicação: 
- petal length (cm) - 4.5 
- petal width (cm) - 1.5 
- sepal length (cm) - 6.0 

Instância 4:
Classe verdadeira (binária): 0
PI-Explicação: 
- petal length (cm) - 4.8 
- petal width (cm) - 1.4 
- sepal width (cm) - 2.8 

Instância 5:
Classe verdadeira (binária): 1
PI-Explicação: 
- petal length (cm) - 1.5 
- petal width (cm) - 0.4 

Instância 6:
Classe verdadeira (binária): 0
PI-Explicação: 
- petal length (cm) - 3.6 
- petal width (cm) - 1.3 
- sepal length (cm) - 5.6 

Instância 7:
Classe verdadeira (binária): 0
PI-Explicação: 
- petal 

In [15]:
# Imprima quantidade em que as features aparecem na explicacão em ordem
ordenando = sorted(contagem_features.items(), key=lambda item: item[1], reverse=True)
for nome_feature, contagem in ordenando:
    print(f"Feature: {nome_feature} Contagem: {contagem}")

Feature: spread1 Contagem: 39
Feature: D2 Contagem: 39
Feature: MDVP:Fo(Hz) Contagem: 39
Feature: MDVP:Fhi(Hz) Contagem: 38
Feature: MDVP:Shimmer(dB) Contagem: 36
Feature: RPDE Contagem: 36
Feature: HNR Contagem: 36
Feature: MDVP:Flo(Hz) Contagem: 36
Feature: PPE Contagem: 36
Feature: spread2 Contagem: 36
Feature: DFA Contagem: 36
Feature: MDVP:APQ Contagem: 36
Feature: MDVP:Shimmer Contagem: 36
Feature: Shimmer:DDA Contagem: 36
Feature: NHR Contagem: 36
Feature: Shimmer:APQ5 Contagem: 36
Feature: Jitter:DDP Contagem: 36
Feature: Shimmer:APQ3 Contagem: 36
Feature: MDVP:Jitter(%) Contagem: 36
Feature: MDVP:RAP Contagem: 36
Feature: MDVP:PPQ Contagem: 36
Feature: MDVP:Jitter(Abs) Contagem: 36
